## Jigsaw Kaggle Competition - WIP 

### Proper commenting and explanation still missing

### Link to google Drive

In [0]:
# Code to read csv file into Colaboratory:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#### Intro and agenda


### Import relevant packages

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
import re
sns.set()
# To release RAM import gc collector
import gc
gc.enable()

In [0]:
import gensim


In [4]:
!pip install emoji


In [0]:
#workaround for zeno
#%run 'emoji/emoji/unicode_codes.py'
#%run 'emoji/emoji/core.py'

### Load Datasets

In [0]:
dtypesDict_tr = {
'id'                            :         'int32',
'target'                        :         'float16',
'severe_toxicity'               :         'float16',
'obscene'                       :         'float16',
'identity_attack'               :         'float16',
'insult'                        :         'float16',
'threat'                        :         'float16',
'asian'                         :         'float16',
'atheist'                       :         'float16',
'bisexual'                      :         'float16',
'black'                         :         'float16',
'buddhist'                      :         'float16',
'christian'                     :         'float16',
'female'                        :         'float16',
'heterosexual'                  :         'float16',
'hindu'                         :         'float16',
'homosexual_gay_or_lesbian'     :         'float16',
'intellectual_or_learning_disability':    'float16',
'jewish'                        :         'float16',
'latino'                        :         'float16',
'male'                          :         'float16',
'muslim'                        :         'float16',
'other_disability'              :         'float16',
'other_gender'                  :         'float16',
'other_race_or_ethnicity'       :         'float16',
'other_religion'                :         'float16',
'other_sexual_orientation'      :         'float16',
'physical_disability'           :         'float16',
'psychiatric_or_mental_illness' :         'float16',
'transgender'                   :         'float16',
'white'                         :         'float16',
'publication_id'                :         'int8',
'parent_id'                     :         'float32',
'article_id'                    :         'int32',
'funny'                         :         'int8',
'wow'                           :         'int8',
'sad'                           :         'int8',
'likes'                         :         'int16',
'disagree'                      :         'int16',
'sexual_explicit'               :         'float16',
'identity_annotator_count'      :         'int16',
'toxicity_annotator_count'      :         'int16'
}

In [0]:
link1 = 'https://drive.google.com/open?id=1JYgyZh1WxXglHQ--rXWtrIZImLxpkNGo'
link3 = 'https://drive.google.com/open?id=1FB8nq7o0zn6lEcqfT_CDetYfvbmxq30a'

fluff1, id1 = link1.split('=')
fluff3, id3 = link3.split('=')

downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('test.csv')  
test_data = pd.read_csv('test.csv')

downloaded = drive.CreateFile({'id':id3}) 
downloaded.GetContentFile('train.csv')  
train_data = pd.read_csv('train.csv',dtype=dtypesDict_tr,parse_dates=['created_date'], nrows=500000 )

del link1, fluff1, id1, fluff3, id3, dtypesDict_tr 

In [8]:
# kill all other columns except comment text
cols_to_keep = ['comment_text','target']
train_data.drop(train_data.columns.difference(cols_to_keep), axis=1)
gc.collect()

817


## Text preprocessing

### Methods

In [9]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

''' Extract all unique characters from the dataset ''' 

def extract_used_characters(df):
    import string
    used_characters = set()
    for i, row in enumerate(df):
        characters = list(row)
        for x in characters:
            used_characters.add(x)
    used_characters = [c for c in used_characters if c not in list(string.ascii_lowercase)]
    return used_characters

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#Remove all remaining symbols
# Some preprocesssing that will be common to all the text classification methods you will see. 
#https://mlwhiz.com/blog/2019/01/17/deeplearning_nlp_preprocess/


"""
The next function eliminates all signs that have been included in the previous 
defined list "puncts". The sign is simply replaced by a whitespace.
"""

def clean_text(text):
    for x in text:
        if x in puncts:
            text = text.replace(x, ' ')
    return text

In [0]:
'''Extract all unique contradictions from the dataset 
including "-" or "'" inside a word''' 

def extract_contractions(df):
    contractions = set()
    for i, row in enumerate(df):
        words = word_tokenize(row)
        for w in words:
          #used_words.add(w)
          if re.match(r"\b\w*[-']\w*\b", w):
            contractions.add(w)
                
    return contractions

In [0]:
"""
This functions check's whether a word contains a contraction or not. 
In case a contraction is found, the corrected value from the dictionary is 
returned
"""

def replace_contractions(text):
    for x in text.split():
        if x in contraction_dict.keys():
            text = text.replace(x, contraction_dict.get(x))
    return text

In [0]:
"""
This function uses the google translator to detect comments that aren't
english. If a comment is found it tries to detect the language and then
translates it to english.
"""

from googletrans import Translator

def translator(text, tlator):
  try:
      language = tlator.detect(text).lang
      if language != 'en':
          text = tlator.translate(text, dest='en')
  except:
      None
  return text

In [0]:
#https://mlwhiz.com/blog/2019/01/17/deeplearning_nlp_preprocess/
"""
The following function is used to format the numbers.
"""

def clean_numbers(x):
    if bool(re.search(r'\d+', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords = set(stopwords.words('english'))


''' Remove stopwords and multiple whitespaces around words'''

def remove_stopwords(text):
  stopword_re = re.compile(r'\s*\b(' + r'|'.join(stopwords) + r')\b\s*')
  text = stopword_re.sub(' ', text)
  text = re.sub(r"\s+"," ", text, flags = re.I)
  text = re.sub(r"^\s+", "", text)
  text = re.sub(r"\s+$", "", text)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Spellchecker

In [0]:
def build_vocab(df):
  
  '''Build a dictionary of words and its number of occurences from the data frame including all words which are not known to nlp vocabluary.
  This filters out typical names of persons, locations etc. which are not known to common language dictionaries'''
  
  vocab = {}
  for i, row in enumerate(df):
      words = word_tokenize(row)
      #words = [word for word in row.split(' ')] pot. faster
      for w in words:
        if nlp.vocab.has_vector(w):
          pass
        else:
          try:
              vocab[w] += 1
          except KeyError:
              vocab[w] = 1
  return vocab

In [17]:
# This comes from CPMP script in the Quora questions similarity challenge. 
from collections import Counter
import heapq
nltk.download('punkt')
from operator import itemgetter
from multiprocessing import Pool


def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
''' This method can be used to replace misspelled words using regex'''
#Sample dict:
#mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
  
  ''' Compile all mispellings (keys of the dictionary) to one regex search 
  function and return the dictionary and the related regex function'''
  
  mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
  return mispell_dict, mispell_re


def replace_typical_mispell(text, mispell_dict):
  ''' Replace all mispellings in a text from the mispelling dictionary '''
  mispellings, mispellings_re = _get_mispell(mispell_dict)
  def replace(match):
      return mispellings[match.group(0)]
  return mispellings_re.sub(replace, text)

### Analysis

In [21]:
# UExtract all unique characters from the comment text as a basis for translation and/or replacement
characters = extract_used_characters(train_data['comment_text'])
print(characters)
del characters
gc.collect()

['”', '️', '⁉', '🐮', '♭', '♍', '‑', '🎲', 'ğ', '会', '😵', 'é', 'ᴍ', '\x08', 'آ', '😃', '🎄', '세', '💃', 'ʊ', '–', 'ם', 'î', 'E', '🐇', '´', '7', '✔', 'ᴀ', 'غ', '😔', 'ᾶ', 'ﬁ', '👤', '्', '💨', '戦', '하', '是', '🌯', '₃', '❗', '𝓿', '\\', '`', 'ŏ', '✬', 'Ì', 'ġ', '\t', '÷', '«', 'п', 'ș', 'ع', 'ツ', 'ἴ', 'ū', 'ن', 'つ', 'S', 'A', '土', '人', 'ó', 'َ', '\xad', '😠', 'τ', 'ل', 'ɪ', '⏏', 'ɡ', 'ᴠ', '∙', 'ὶ', '━', '🗑', '😂', '′', '🇼', '\u200b', 'Ï', 'כ', 'α', '😲', '文', '🐾', '😰', 'σ', '🌎', '🔤', '🍺', 'R', '༼', 'ć', '🏿', '你', 'ệ', 'ᴊ', '‐', '圆', '不', '†', '👅', '½', '▄', 'έ', '🏡', '𝘳', 'ʌ', '\u2008', '🍔', ':', '\u200a', '́', '🏻', '‼', '💞', '坏', '😛', '💜', 'ン', 'ü', 'P', '𝘴', '\x7f', '☭', '\x95', '♡', '🍕', 'ῃ', 'K', '▒', 'ύ', '✒', 'Z', '😈', '\u2004', '♪', '\u2002', 'â', ';', '—', '$', '◐', 'Ｓ', '▱', '℅', '☔', '💚', '🤐', '8', 'μ', '有', '’', '😢', 'ю', 'Ж', '\x96', 'т', '🙏', '̶', 'D', 'ة', '𝘮', '🤡', '⁄', '😊', '👄', '英', 'с', 'ɒ', '🚌', 'ë', '🎵', '…', '☆', '%', '¡', '┗', 'Ｐ', 'É', 'ʃ', 'γ', '¢', 'è', '"', '۞', '🏈', 'ὁ', '😣

576

The extraction of all characters shows that a wide list of symbols, foreign and emojies are being used. As a major part of them representing a certain meaning.
The following steps will step by step translate the symbols into meaningful words.

Firstly, emojies are translated into words using an out of the box library.

In [0]:
# Import the emoji library for translation
import emoji
# Translate the emojies into common words
train_data.comment_text = train_data.comment_text.apply(lambda x: emoji.demojize(x))
del emoji

Based on the EDA we saw that most of the comments are in English. However, the closer inspection of the symbols shows that the dataset contains Chinese or Arabic letters. We will thus translate the comments.

In [0]:
# Initiate the translator
transl = Translator()
# Translate the comments which are not in english
train_data.comment_text = train_data.comment_text.apply(lambda x: translator(x, transl))

In [0]:
# UExtract all unique characters from the comment text as a basis for translation and/or replacement
other_characters = extract_used_characters(train_data['comment_text'])
other_characters.remove(' ')
print(other_characters)

 ? : ɡ ° € 群 ר ʸ » ا 会 ؟ ᴺ ▄ 𝓒 г 1  ю 殊 ʲ ɑ 
 è ᴅ π 𝘰 ｒ ل د 歌 ʳ ɛ 你 ☆ ♡ ы ﻿ ғ ​ 土 † α ἔ ק ! א ₃ о 华 7 温 ̅ 加 ѕ ☭ ć 是 к 故 ₂  ▓ ö מ 𝘮 K ł ᴠ ﬂ х ▰ œ ’ ß O ༼ л ε ¾  ي 戦 圆 ر ž έ ｍ 文 ｐ 园 하 ê ġ ἐ ‪ ▇ 6 س $ ō “ ‎ 特 ὐ ` 3 ; ʻ ❧ ṣ َ 	 ́ в ń ˙ ì ｏ ༽ 子 ᴋ   ℅ ＼ 𝘵 ♫ ツ ο ř ｃ φ æ ʼ љ ى ۩ " ʜ 坏 μ ！  ・ ­ أ 降 ك ” ✬ ▂  н ▆ ℐ ע ŏ ʌ レ % ▅ û ή 세 ′ म ͝ 🖑 ћ ב ´ , / ã \ ᴏ  2 ¡ δ ¢ U . ̲ サ β я ₄ 钱    ┣   ⁴ ➤ п 系   ★ – 》 ｓ ： 4  ► ʷ ² ⋅ ♪ 失 и ̶ 𝓿 议 ɒ 不   ᴜ י ‐ ╪ ˌ ᵒ ▔ ∞ … • ī −  ͞ ᴇ 骗 ɔ 요 ä ░ ½ ▃ º 有 ʰ ν ά ═ ʃ █ ≠ µ   ệ ό 舞 ｔ ת ¯  ᴵ 地 е › ت W   ￼ 关 谷   γ ς ق 成 8 ｈ 𝘺 ὲ ך ه ò ף  小 ט з 别 ē 去 ± ◞ ₁ ━ ל ▬ ô ة ü ə | ‏ 蔡 V ط ◕ ι ц { ⅓ ф ┫ — ᴊ ш 　 ч   ˢ с ع 伎 ę ح ģ · ʟ ж щ 𝘦 ư ś б ｅ 哥  क ● ║ ĸ 𝘳 м ‑ エ „ ơ £ ᴡ ω و ‬ ᵗ 拿 ь â ἴ ン ̈ ā 9 σ   つ ἀ  ἰ ा ș 税  ف ┗ ύ 障  إ < צ ⇒ 

In [0]:
# Extract contradictions from the dataset
contradictions = extract_contractions(train_data['comment_text'])
del contradictions
gc.collect()

##### Spellchecking with Spacy and GoogleNews

In [0]:
# pip install spacy
import spacy
!python -m spacy download en_core_web_lg
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_lg")

# Build the vocabluary and delete the big datasets afterwards
vocab = build_vocab(train_data['comment_text'])
del nlp, spacy
gc.collect()

328

In [0]:
# Load GoogleNews vocabluary to the disk
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [0]:

# Assign the embedding Word2Vec from Google News and load the model
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
model = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, 
                                                        binary=True)
del EMBEDDING_FILE

# Extract the words from the model
words = model.index2word

# Rank the words of the vocabluary
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [0]:
# Select the top occuring words from unknown dataset
top_words = dict(heapq.nlargest(1000, vocab.items(), key=itemgetter(1)))
print(top_words)
del vocab

In [0]:
# Map the word correction to the top mispelled words
pool = Pool(4)
corrected_words = pool.map(correction,list(top_words.keys()))

In [0]:
# generate the missell dict
mispell_dict = {}
for word,corrected_word in zip(top_words,corrected_words):
    if word!=corrected_word:
        mispell_dict[word] = corrected_word

In [0]:
# Show the library
mispell_dict

In [0]:
# Replace mispellings according to the dictionary
train_data.comment_text = train_data.comment_text.apply(lambda x: replace_typical_misspell(x))


In [0]:
# Delete nun-needed libraries
del mispell_dict, WORDS, words, w_rank, model, pool,Pool, corrected_words, itemgetter

gc.collect()

0

In [0]:
#%whos

### Visualize most common words

from nltk.corpus import stopwords
nltk.download('stopwords')
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize


stop_words = stopwords.words('english')


-# Tokenize the comment text
tok_comments = [word_tokenize(com) for com in train_data.comment_text]


-# Remove stopwords
tokens = [[w for w in s if (w not in stop_words) & (len(w)>2)] for s in tok_comments]

from nltk.probability import FreqDist

-#plot the most frequent words
tokens = np.array([np.array(s) for s in tokens])
fdist = FreqDist(np.concatenate(tokens))
fdist.plot(30,cumulative=False)
plt.show()

-#plot the most frequent word pairs
from nltk import bigrams, ngrams
bigrams_tokens = bigrams(np.concatenate(tokens))
fdist_bigrams = FreqDist(list(bigrams_tokens))
fdist_bigrams.plot(30,cumulative=False)
plt.show()

from collections import Counter

from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(tok_comments)

dictionary.token2id

corpus = [dictionary.doc2bow(com) for com in tok_comments]

## Feature engineering

In [0]:
from textblob import TextBlob

# Make use of the build in Text Blob sensitivity features to add them towards a model
train_data['polarity'] = train_data['comment_text'].apply(lambda com: TextBlob(com).sentiment.polarity)
train_data['subjectivity'] = train_data['comment_text'].apply(lambda com: TextBlob(com).sentiment.subjectivity)
del TextBlob

## Count vectorizer

#### Vectorize

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


In [0]:
# Separate the target variable into a binary output variable to fit to logistic regression and other clasification models
y = np.where(train_data.target>0.5,1, 0 )

Using count vectorizer

In [0]:
# Initiate the count vectorizer
count_vectorizer = CountVectorizer(stop_words='english') #, max_features=1000
# Create a count_vectorizer for out dataset
count_data = count_vectorizer.fit_transform(train_data.comment_text.values)


In [0]:
#count_data_arr = count_data.toarray()

In [0]:
#count_data_df = pd.DataFrame(count_data_arr)

In [0]:
#del count_vectorizer

In [0]:
#add_features = ['polarity', 'subjectivity']
#count_data = pd.concat([count_data_df,train_data.loc[:, add_features]], axis=1)

In [0]:
# Do we need scaling??

# Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#count_data = sc.fit_transform(count_data)

In [0]:
# Do a train-test-split on the counted data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(count_data, y, test_size=0.2, random_state=42)

#### Logistic regression

In [0]:
# Fit a first logictic regression model
log = LogisticRegression()
log.fit(X_train_c, y_train_c)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
# Predict on the text_set
pred = log.predict(X_test_c)
pred_prob = log.predict_proba(X_test_c)

In [0]:
#Get the accuracy
metrics.accuracy_score(y_test_c, pred)

0.95744

In [0]:
# Get the ROC AUC score
metrics.roc_auc_score(y_test_c, pred)

0.7337503728242796

### Light GBM  - to be adapted to non binary Y variable, and loss function


In [0]:
#https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y_train)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary' # ADAPT
params['metric'] = 'binary_logloss' # ADAPT
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

clf = lgb.train(params, d_train, 100)

In [0]:
#Prediction
y_pred=clf.predict(x_test)

#convert into binary values

for i in range(0,99):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [0]:
#Confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

#Accuracy

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

Parameter Tuning:

Data scientists always struggle in deciding when to use which parameter? and what should be the ideal value of that parameter?

Following set of practices can be used to improve your model efficiency.

    num_leaves: This is the main parameter to control the complexity of the tree model. Ideally, the value of num_leaves should be less than or equal to 2^(max_depth). Value more than this will result in overfitting.
    min_data_in_leaf: Setting it to a large value can avoid growing too deep a tree, but may cause under-fitting. In practice, setting it to hundreds or thousands is enough for a large dataset.
    max_depth: You also can use max_depth to limit the tree depth explicitly.

For Faster Speed:

    Use bagging by setting bagging_fraction and bagging_freq
    Use feature sub-sampling by setting feature_fraction
    Use small max_bin
    Use save_binary to speed up data loading in future learning
    Use parallel learning, refer to parallel learning guide.

For better accuracy:

    Use large max_bin (may be slower)
    Use small learning_rate with large num_iterations
    Use large num_leaves(may cause over-fitting)
    Use bigger training data
    Try dart
    Try to use categorical feature directly

To deal with over-fitting:

    Use small max_bin
    Use small num_leaves
    Use min_data_in_leaf and min_sum_hessian_in_leaf
    Use bagging by set bagging_fraction and bagging_freq
    Use feature sub-sampling by set feature_fraction
    Use bigger training data
    Try lambda_l1, lambda_l2 and min_gain_to_split to regularization
    Try max_depth to avoid growing deep tree

Conclusion:

I implemented LightGBM on multiple datasets and found that its accuracy challenged other boosting algorithms. From my experience, I will always recommend you to try this algorithm at least once.

I hope you guys enjoyed this blog and it was useful to all. I would request you you to give suggestion which will help to improve this blog.

Source: Microsoft LightGBM Documentation

## TFIDF vectorizer

#### Vectorize

In [0]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_data = tfidf_vectorizer.fit_transform(train_data.comment_text.values)

In [0]:
tfidf_data

<500000x129727 sparse matrix of type '<class 'numpy.float64'>'
	with 10555981 stored elements in Compressed Sparse Row format>

In [0]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(tfidf_data, y, test_size=0.2, random_state=42)

#### Logistic regression

In [0]:
log = LogisticRegression()
log.fit(X_train_t, y_train_t)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
pred = log.predict(X_test_t)

In [0]:
metrics.accuracy_score(y_test_t, pred)

0.95955

In [0]:
metrics.roc_auc_score(y_test_t, pred)

0.7040973935998638

#### !! Use spacy to maybe correct only non-names

## Word2Vec

In [0]:
from gensim.models import Word2Vec

In [0]:

# Tokenize the train data based on spaces
tokenized_corpus = [[w for w in document.split(' ')] for document in train_data['comment_text']]


[['cool',
  'like',
  'would',
  'want',
  'mother',
  'read',
  'really',
  'great',
  'idea',
  'well',
  'done',
  ''],
 ['thank',
  'would',
  'make',
  'life',
  'lot',
  'le',
  'anxiety',
  'inducing',
  'keep',
  'let',
  'anyone',
  'get',
  'way',
  ''],
 ['urgent', 'design', 'problem', 'kudos', 'taking', 'impressive', ''],
 ['something', 'able', 'install', 'site', 'releasing', ''],
 ['haha', 'guy', 'bunch', 'loser', '']]

In [0]:
# Set values for various parameters
feature_size = 100    # Word vector dimensionality  
window_context = 5          # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

#Create a word2vec model
w2v_model = Word2Vec(tokenized_corpus, size=feature_size, 
                              window=window_context, min_count = min_word_count,
                              sample=sample, iter=100)

####### visualize embeddings
from sklearn.manifold import TSNE

words = w2v_model.wv.index2word
wvs = w2v_model.wv[words]
tsne = TSNE(n_components=2, random_state=0, n_iter=250, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(wvs)
labels = words

plt.figure(figsize=(12, 6))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
  plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

In [0]:
''' Average the word vectors to get a vector per document '''

def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
   
def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# get document level embeddings
w2v_feature_array = averaged_word_vectorizer(corpus=tokenized_corpus, model=w2v_model,
                                             num_features=feature_size)
#pd.DataFrame(w2v_feature_array)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [0]:
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(w2v_feature_array, y, test_size=0.2, random_state=42)

#### Logistic regression

In [0]:
log = LogisticRegression()
log.fit(X_train_t, y_train_t)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
pred = log.predict(X_test_t)

In [0]:
metrics.accuracy_score(y_test_t, pred)

0.95955

In [0]:
metrics.roc_auc_score(y_test_t, pred)

0.7040973935998638

##### So far model quality very bad comparing to the proportion

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
    
    Build a neural network with embeddings, using Glove, 
    in the last dense leyer we should use sigmoid function!!

Try also LightGBM, Naive Bayes, CNN, RNN